# Cuaderno 5: Perceptrones

Los *perceptrones* fueron introducidos por Frank Rosenblatt entre 1957 y 1962 como una familia de modelos teóricos y experimentales de redes neuronales artificiales. En su momento generaron mucho entusiasmo, pero también bastante controversia sobre sus alcances. A partir de ese trabajo surgieron modelos posteriores más abstractos y matemáticamente formales.  

El antecedente directo fue la **neurona de McCulloch y Pitts** (1943), que proponía una neurona muy simplificada, pensada como un elemento de cálculo lógico. Esa idea abrió la puerta a imaginar redes de unidades elementales capaces de computar funciones más complejas.  

Hoy en día, cuando decimos *perceptrón simple* solemos referirnos a un clasificador lineal de una sola capa (con una función de activación tipo escalón o sigmoide), y con *perceptrón multicapa* (MLP, *multilayer perceptron*) a una red de retroalimentación (*feedforward*) con al menos una capa oculta y entrenada mediante retropropagación (un algoritmo que recién se popularizó en los años 80).  

Aunque la inspiración inicial vino de la biología, las redes neuronales modernas abstraen mucho esos detalles y se piensan más como funciones matemáticas: cada capa aplica una transformación lineal seguida de una no linealidad. En ese sentido, comparten gran parte de la matemática con la **regresión logística**.  

La diferencia es que las redes neuronales son más poderosas: se ha demostrado que un perceptrón multicapa con una sola capa oculta y suficientes neuronas puede aproximar cualquier función continua en un dominio acotado (esto se conoce como el **teorema de aproximación universal**).


## Configuración

Como es usual, comenzamos importando las librerías que vamos a utilizar. Corré las celdas en esta sección para cargar las funciones que vamos a usar en el cuaderno.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Funciones de graficado

In [ ]:
def visualizar_errores(t, errors, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(t, errors)
    ax.set_ylim(-2, 2)
    ax.set_xlabel("Iteración")
    ax.set_ylabel("Error")
    ax.axhline(0, color='r')

def visualizar_pesos(W, b, X=None, Y=None, ax=None, titulo="Límites de decisión"):
    """
    W: (2,) o (k,2)  pesos [w1, w2] por neurona
    b: escalar o (k,1)/(k,)  sesgo por neurona
    X, Y: opcionales para plotear puntos del dataset
    """
    if ax is None:
        fig, ax = plt.subplots()

    # Normalizar formas
    W = np.atleast_2d(W)           # -> (k,2)
    if np.isscalar(b):
        b = np.array([b]*len(W)).reshape(-1, 1)  # -> (k,1)
    else:
        b = np.atleast_2d(b)
        if b.shape[0] != W.shape[0]:
            # si vino transpuesto, intentamos acomodar
            if b.shape[1] == W.shape[0]:
                b = b.T
        if b.shape[1] != 1:
            b = b.reshape(-1, 1)

    # Rango amigable para entradas binarias
    x_vals = np.linspace(-0.2, 1.2, 400)

    # Dibujar una recta por neurona: w1*x + w2*y + b = 0  -> y = -(w1*x + b)/w2
    for i, (wi, bi) in enumerate(zip(W, b)):
        if abs(wi[1]) > 1e-12:
            y_vals = -(wi[0]*x_vals + bi.item()) / wi[1]
            ax.plot(x_vals, y_vals, linestyle='--', label=f"neurona {i+1}")
        else:
            # frontera vertical: x = -b/w1
            x0 = -bi.item() / wi[0]
            ax.axvline(x0, linestyle='--', label=f"neurona {i+1}")

    # Puntos del dataset (si se pasan)
    if X is not None and Y is not None:
        yy = Y.ravel()
        ax.scatter(X[0, yy==0], X[1, yy==0], marker='o', label='Clase 0')
        ax.scatter(X[0, yy==1], X[1, yy==1], marker='s', label='Clase 1')

    ax.set_xlim(-0.2, 1.2)
    ax.set_ylim(-0.2, 1.2)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(titulo)
    ax.legend(loc='upper right', frameon=False)

## Perceptrón simple

El primer problema de nuestro perceptrón va a ser el de aprender a imitar una compuerta lógica **OR**.  
Las compuertas OR son dispositivos electrónicos con una función booleana que devuelve 1 cuando al menos una de sus dos entradas está prendida.  

El objetivo del perceptrón va a ser, a partir de dos entradas binarias y una salida esperada, ajustar los pesos de su capa de activación de manera que, tras entrenarse varias veces, siempre pueda reproducir la función booleana OR.  

En este caso, vamos a entrenar un perceptrón de dos nodos de entrada y uno de salida con los siguientes pares de ejemplo:

$$
\begin{split}
\mathbf{X}_1 = \begin{pmatrix} 0 \\ 0 \end{pmatrix}, \; \mathbf{Y}_1 = \begin{pmatrix} 0 \end{pmatrix} \\
\mathbf{X}_2 = \begin{pmatrix} 0 \\ 1 \end{pmatrix}, \; \mathbf{Y}_2 = \begin{pmatrix} 1 \end{pmatrix} \\
\mathbf{X}_3 = \begin{pmatrix} 1 \\ 0 \end{pmatrix}, \; \mathbf{Y}_3 = \begin{pmatrix} 1 \end{pmatrix} \\
\mathbf{X}_4 = \begin{pmatrix} 1 \\ 1 \end{pmatrix}, \; \mathbf{Y}_4 = \begin{pmatrix} 1 \end{pmatrix}
\end{split}
$$

Para expresarlo de manera más compacta, podemos reunir todas las combinaciones en una matriz de entradas $\mathbf{X}$ y un vector de salidas $\mathbf{Y}$:

$$
\mathbf{X} =
\begin{pmatrix}
0 & 0 & 1 & 1 \\
0 & 1 & 0 & 1
\end{pmatrix}, \quad
\mathbf{Y} =
\begin{pmatrix}
0 & 1 & 1 & 1
\end{pmatrix}
$$

De esta manera el perceptrón ve **todas las posibles combinaciones** de las dos entradas y la salida esperada correspondiente.

### Componentes del perceptrón

Un perceptrón simple tiene tres elementos básicos:

- **Entradas:** los valores que alimentan al perceptrón (acá, dos bits de entrada).  
- **Pesos:** cada entrada tiene un peso asociado que indica su importancia relativa. Estos pesos se ajustan durante el entrenamiento.  
- **Función de activación:** después de calcular la suma ponderada $z$, se aplica una función que decide si la neurona se “activa” o no.

### Modelo matemático

Podemos modelar los pesos con un vector columna $\mathbf{w} = (w_1, w_2)^\top$.  
Para un ejemplo de entrada $\mathbf{x} = (x_1, x_2)^\top$, la salida lineal del perceptrón es:

$$
z = \mathbf{w}^\top \mathbf{x} + b
$$

donde $b$ es el **sesgo**, que permite desplazar la frontera de decisión.

Finalmente, este valor $z$ se pasa por una función de activación no lineal. En el perceptrón original de Rosenblatt se usaba una **función escalón** (devuelve 0 o 1). En versiones modernas se usa a menudo la **función sigmoide**:

$$
\hat{y} = \sigma(w_1 x_1 + w_2 x_2 + b)
$$

donde $\sigma$ es la función sigmoide logística. El valor $\hat{y}$ es la salida del perceptrón y se compara con la salida esperada $Y$ para ajustar los pesos durante el entrenamiento.

### Función de activación $\sigma$

La **función de activación** transforma el resultado de multiplicar las entradas por los pesos y sumar el sesgo en un valor no lineal. Esto es clave cuando queremos que el perceptrón tome una **decisión binaria**, como en problemas de clasificación.  

Una función de activación muy usada es la **sigmoide** (aunque hay muchas otras):

$$
\sigma(x) = \frac{1}{1 + e^{-\beta x + b}}
$$

Esta función devuelve valores en el rango $(0,1)$, nunca exactamente 0 o 1, pero muy cercanos. Así podemos interpretar la salida como una **probabilidad** de que la neurona se active.  

Ejecutá la celda siguiente para verla en acción, y pensá:  

- ¿Qué ocurre cuando aumenta $\beta$?

In [ ]:
def sigmoid(x, beta, b):
    return 1 / (1 + np.exp(-beta * x + b))

x = np.arange(-10, 10, 0.1)

@widgets.interact(beta=(0, 10, 0.01), b=(-10, 10, 0.5))
def simulate(beta, b=0):
    plt.plot(x, sigmoid(x, beta, b))
    plt.vlines(x=0, ymin=0, ymax=1, color='r', linestyles='--')

### Compuerta OR

El entrenamiento del perceptrón se realiza utilizando un algoritmo conocido como regla de aprendizaje del perceptrón. Este ajusta los pesos basándose en el error de las predicciones, buscando minimizar la diferencia entre las salidas predichas y las reales. En la clase vimos como combinando el aprendizaje Hebbiano con la regla de la cadena podemos ir aproximandonos a los pesos que buscamos.

Veamos la implementación de un perceptrón simple y entrenémoslo para detectar la compuerta OR:

In [ ]:
# Umbral de activación
threshold = 0.5

# Definimos el modelo lineal que queremos resolver
def z(X, W, b):
    return W.dot(X) + b

# Definimos la función de activación
def sigma(z):
    return 1 / (1 + np.exp(-z))

def y(X, W, b):
    return sigma(z(X, W, b))

# Definimos la derivada de la función de activación dy/dw
def dg(y):
    return y * (1 - y)

# Definimos la función que calcula el error
def loss(x, x_pred):
    return x - x_pred

# Definimos la funcion que entrena un conjunto de datos de entrada y salida y grafica el error cometido
# en cada salto de tiempo.
# X: Las entradas del conjunto de entrenamiento
# Y: Las salidas esperadas del conjunto de entrenamiento
# alpha: Constante de aprendizaje
# tmax: Tiempo máximo que queremos entrenar
def perceptron_simple(X, Y, alpha=0.5, tmax=1000):
    # Iniciamos un array que va a guardar los errores en cada paso para poder graficarlos al final
    # y observar como va aprendiendo el perceptrón en cada paso
    errors = []

    # Fijamos la semilla para reproducir siempre los mismos valores aleatorios
    np.random.seed(1) 
    
    # Iniciamos la matriz con los pesos en forma aleatoria
    W = np.random.randn(Y.shape[0], X.shape[0])

    # Iniciamos el término de sesgo con un número aleatorio
    b = np.random.uniform(-1, 1)
    
    # Definimos la cantidad de trials que vamos a usar
    t = np.arange(tmax)

    for step in t:
        y_hat = y(X, W, b)
        error = loss(Y, y_hat)
        delta = error * dg(y_hat)
        W = W + alpha * delta.dot(X.T)
        b = b + alpha * delta.sum()

        errors = np.append(errors, np.sum(error ** 2))

    # Graficamos el error
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
    visualizar_errores(t, errors, ax1)
    visualizar_pesos(W, b, X, Y, ax2)
    plt.show()

    # Imprimimos los parámetros
    print("W =", W)
    print("b =", b)
    
    return lambda X: np.where(y(X, W, b) < threshold, 0, 1)

# Entrenamos al perceptrón para la funcion OR
@widgets.interact(alpha=(0, 1, 0.01), tmax=(1, 1000))
def simulate(alpha=0.5, tmax=1000):
    X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
    Y = np.array([[0, 1, 1, 1]])
    perceptron_simple(X, Y, alpha, tmax)

Ejercitamos al perceptrón con dos valores de entrada:

In [ ]:
X = np.array([[0, 0, 1, 1], 
              [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 1]])
f = perceptron_simple(X, Y, alpha=0.1, tmax=100)
y_hat = f(np.array([[1], [0]]))
print(f"Salida: {y_hat}")

### Compuerta AND

En una forma similar, podemos entrenar al perceptrón para detectar cuando los dos nodos de entrada se activan al mismo tiempo. Esto puede ser visto como la función booleana AND. En este caso, los vectores de entrenamiento y de salida esperada pueden ser:

$$\begin{split}
\pmb{X_1} = \begin{pmatrix} 0 \\ 0 \end{pmatrix} & \pmb{Y_1} = \begin{pmatrix} 0 \end{pmatrix} \\
\pmb{X_2} = \begin{pmatrix} 0 \\ 1 \end{pmatrix} & \pmb{Y_2} = \begin{pmatrix} 0 \end{pmatrix} \\
\pmb{X_3} = \begin{pmatrix} 1 \\ 0 \end{pmatrix} & \pmb{Y_3} = \begin{pmatrix} 0\end{pmatrix} \\
\pmb{X_4} = \begin{pmatrix} 1 \\ 1 \end{pmatrix} & \pmb{Y_4} = \begin{pmatrix} 1 \end{pmatrix}
\end{split}$$

Para hacerlo con nuestro código, no tenemos más que entrenar al perceptrón con las nuevas matrices:

In [ ]:
X = np.array([[0, 0, 1, 1], 
              [0, 1, 0, 1]])
Y = np.array([[0, 0, 0, 1]])
f = perceptron_simple(X, Y, alpha=0.5, tmax=1000)
y_hat = f(np.array([[1], [0]]))
print(f"Salida: {y_hat}")

### Compuerta XOR

Veamos ahora la compuerta XOR. Esta compuerta, también conocida como compuerta OR exclusiva, detecta cuando una entrada esta activa, pero no la otra. Para resolver este caso, los vectores de entrenamiento y de salida esperada pueden ser:

$$\begin{split}
\pmb{X_1} = \begin{pmatrix} 0 \\ 0 \end{pmatrix} & \pmb{Y_1} = \begin{pmatrix} 0 \end{pmatrix} \\
\pmb{X_2} = \begin{pmatrix} 0 \\ 1 \end{pmatrix} & \pmb{Y_2} = \begin{pmatrix} 1 \end{pmatrix} \\
\pmb{X_3} = \begin{pmatrix} 1 \\ 0 \end{pmatrix} & \pmb{Y_3} = \begin{pmatrix} 1 \end{pmatrix} \\
\pmb{X_3} = \begin{pmatrix} 1 \\ 1 \end{pmatrix} & \pmb{Y_4} = \begin{pmatrix} 0 \end{pmatrix}
\end{split}$$

Este problema de apariencia inocente es difícil porque no es linealmente separable. Si ponemos las cuatro posibles entradas en un plano, no hay ninguna línea recta que separe los casos que tienen que dar 1 de los casos que tienen que dar 0. 
Vamos a comparar a una red de 2 entradas y una salida que entrenamos con la regla delta y a una red que como la presentada más arriba, además de las 2 entradas y la salida tiene dos unidades en la capa oculta.

*Ejercicio: Entrenar al perceptrón con estos vectores de entrenamiento y de salida esperados.*

In [ ]:
# Entrenamos al perceptrón para la funcion XOR
X = np.array([[0, 0, 1, 1], 
              [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])
f = perceptron_simple(X, Y, alpha=0.5, tmax=1000)
y_hat = f(np.array([[1], [0]]))
print(f"Salida: {y_hat}")

No es posible construir un perceptrón de una sola capa que compute la operación lógica **XOR**. En su libro *Perceptrons* (1969), Marvin Minsky y Seymour Papert demostraron varios teoremas sobre las limitaciones de estos modelos, entre ellos que no podían implementar funciones como la exclusión OR (XOR), la paridad o la conectividad.

La intuición es que un perceptrón simple es un **clasificador lineal**. Para dos entradas, la ecuación

$$
w_1 x_1 + w_2 x_2 + b = 0
$$

describe una recta que actúa como frontera de decisión: todos los puntos de un lado se clasifican como 0 y todos los del otro como 1.  

El problema es que la función XOR **no es linealmente separable**: no existe una sola recta que divida el plano en las dos categorías correctas. Para lograrlo hacen falta múltiples rectas combinadas, es decir, **una red con al menos una capa oculta** (un perceptrón multicapa).


## Perceptrón multicapa

Los resultados de Minsky y Papert (1969) se aplicaban únicamente a perceptrones de **una sola capa**.  
En 1974, Paul Werbos presentó en su tesis doctoral un procedimiento general para ajustar adaptativamente los pesos de un sistema no lineal diferenciable, calculando derivadas desde las salidas hacia las entradas.  
Este algoritmo, conocido como **retropropagación del error** (*backpropagation*), permite entrenar perceptrones multicapa mediante descenso por gradiente y muestras de entrenamiento.  
El trabajo se difundió masivamente recién en 1986, gracias a Rumelhart, Hinton y Williams.  

El **perceptrón multicapa (MLP)** agrega una o más **capas ocultas** (\(h\), por *hidden*) conformadas por un número arbitrario de nodos ubicados entre la capa de entrada y la de salida.  
Los nodos de salida ya no reciben directamente las entradas originales, sino las activaciones de la capa oculta.  

Un MLP con una sola capa oculta debe aprender dos matrices de pesos y dos sesgos:  

- \(W_h, b_h\): conectan la entrada con la capa oculta.  
- \(W_y, b_y\): conectan la capa oculta con la salida.  

De esta forma, la salida de la red se define como:

$$
\hat{y} \;=\; \sigma\!\big(W_y \, \sigma(W_h X + b_h) + b_y\big)
$$

donde \(\sigma\) es la función de activación (sigmoide, tanh, ReLU u otra).  
La retropropagación permite ajustar estos parámetros para que la salida \(\hat{y}\) se aproxime cada vez más a la salida esperada.

In [ ]:
# Definimos la funcion que entrena un conjunto de datos de entrada y salida y grafica el error cometido
# en cada salto de tiempo.
# X: Las entradas del conjunto de entrenamiento
# Y: Las salidas esperadas del conjunto de entrenamiento
# nodos_capa_oculta: Cantidad de nodos en la capa oculta
# tmax: Tiempo máximo que queremos entrenar
# alpha: Constante de aprendizaje
def perceptron_multicapa(X, Y, nodos_capa_oculta=2, alpha=0.5, tmax=1000):
    # Iniciamos un array que va a guardar los errores en cada paso para poder graficarlos al final
    # y observar como va aprendiendo el perceptrón en cada paso
    errors = []

    # Fijamos la semilla para reproducir siempre los mismos valores aleatorios
    np.random.seed(1) 

    # Iniciamos la matriz con los pesos de la capa de salida en forma aleatoria
    W_y = np.random.randn(Y.shape[0], nodos_capa_oculta)

    # Iniciamos la matriz con los pesos de la capa oculta en forma aleatoria
    W_h = np.random.randn(nodos_capa_oculta, X.shape[0])

    # Iniciamos los términos de sesgo para la capta de salida y la capa oculta con números aleatorios
    b_y = np.random.uniform(-1, 1)
    b_h = np.random.uniform(-1, 1, size=(nodos_capa_oculta, 1))

    # Definimos la cantidad de trials que vamos a usar
    t = np.arange(tmax)

    for step in t:       
        h = y(X, W_h, b_h)
        h_hat = y(X, W_h, b_h)
        
        y_hat = y(h_hat, W_y, b_y)

        error = loss(Y, y_hat)
        
        delta_y = error * dg(y_hat)
        delta_h = W_y.T.dot(delta_y) * dg(h_hat)
        
        W_h = W_h + alpha * delta_h.dot(X.T)
        W_y = W_y + alpha * delta_y.dot(h_hat.T)
        
        b_y = b_y + alpha * delta_y.sum()
        b_h = b_h + alpha * delta_h.sum(axis=1, keepdims=True)

        errors = np.append(errors, np.sum(error ** 2))

    # Graficamos el error
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
    visualizar_errores(t, errors, ax1)
    visualizar_pesos(W_h, b_h, X, Y, ax2)
    plt.show()

    # Imprimimos los parámetros
    print("W_h =", W_h)
    print("b_h =", b_h)
    print("W_y =", W_y)
    print("b_y =", b_y)

    return lambda X: np.where(y(y(X, W_h, b_h), W_y, b_y) < threshold, 0, 1)

# Entrenamos al perceptrón para la funcion OR, tm
@widgets.interact(nodos_capa_oculta=(1, 8), alpha=(0, 2, 0.1), tmax=(1, 1000))
def simulate(nodos_capa_oculta=2, alpha=0.5, tmax=1000):
    X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
    Y = np.array([[0, 1, 1, 0]])
    perceptron_multicapa(X, Y, nodos_capa_oculta, alpha, tmax)

Ejercitamos la red aprendida con un conjunto de entradas:

In [ ]:
# Entrenamos al perceptrón para la funcion XOR
X = np.array([[0, 0, 1, 1], 
              [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])
f = perceptron_multicapa(X, Y, alpha=0.5, tmax=1000)
y_hat = f(np.array([[1], [0]]))
print(f"Salida: {y_hat}")

Se ha demostrado que las redes neuronales *feedforward* multicapa con un número suficiente de unidades ocultas entre las unidades de entrada y salida tienen una propiedad de aproximación universal: "pueden aproximar prácticamente cualquier función de interés con el grado de precisión deseado" (Hornik et al. 1989).

## El dataset MNIST

Hasta ahora trabajamos con ejemplos muy simples de compuertas lógicas. Para dar un paso más, vamos a aplicar perceptrones al conjunto de datos **MNIST** (*Modified National Institute of Standards and Technology database*), un clásico en el área de aprendizaje automático.  

MNIST contiene **70.000 imágenes en escala de grises de dígitos manuscritos (0–9)**, cada una de **28×28 píxeles**. Es un benchmark muy usado porque es suficientemente grande y variado para poner a prueba modelos de clasificación, pero al mismo tiempo es manejable en una computadora personal.  

En nuestro caso vamos a:  

- Descargar el dataset directamente con `fetch_openml` de `sklearn.datasets`.  
- Normalizar los valores de los píxeles al rango $[0,1]$.  
- Codificar las etiquetas de salida en formato **one-hot**: cada dígito (0–9) se representa como un vector de 10 posiciones con un único 1 en la posición correspondiente.  

De esta manera, $X$ contendrá todas las imágenes como vectores de 784 características (28×28), y $Y$ contendrá los vectores one-hot con las etiquetas de los dígitos.  

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
digits = np.eye(10)
X = np.array(mnist.data / 255.0)
Y = np.array([digits[y] for y in mnist.target.astype(int)])

In [ ]:
print(digits)

In [ ]:
print(X.shape)

### Visualización de los dígitos

Antes de entrenar una red neuronal, conviene explorar el dataset para hacernos una idea de cómo son las imágenes y sus etiquetas.  
Recordemos que cada ejemplo en `X` es un vector de 784 posiciones (los píxeles de una imagen 28×28 puestos en fila), y que la etiqueta correspondiente en `Y` está codificada en formato *one-hot*.  

Para facilitar la inspección, vamos a usar un widget interactivo:

In [ ]:
@widgets.interact(i=(0, len(X)-1))
def visualizar_digito(i=0):
  plt.matshow(X[i].reshape(28, 28), cmap='gray')
  plt.show()
  print("y =", Y[i])

Ejecutá la siguiente celda para definir el perceptrón multicapa. Es código es igual al anterior, con la novedad de que el entrenamiento lo haremos por lotes (_batches_).

In [ ]:
def g(y):
  return sp.special.expit(y)

def dg(a):
  return a * (1.0 - a)

# Definimos la función que calcula el error
def loss(x, x_pred):
  return x - x_pred

def z(X, W, b):
  return X @ W + b

def perceptron_multicapa(X, Y, nodos_capa_oculta=128, tmax=50, alpha=0.01, batch_size=128):
  rng = np.random.default_rng(1)
  
  N, d = X.shape
  k = Y.shape[1]
  H = nodos_capa_oculta

  # Pesos de la capa oculta
  W_h = rng.standard_normal((d, H), dtype=np.float32)

  # Pesos de la capa de salida
  W_y = rng.standard_normal((H, k), dtype=np.float32)
  b_y = np.zeros(k, dtype=np.float32)
  
  # Sesgos de la capa oculta
  b_h = np.zeros(H, dtype=np.float32)

  losses = []
  num_batches = (N + batch_size - 1) // batch_size

  for t in range(tmax):
    idx = rng.permutation(N)
    X_shuf, Y_shuf = X[idx], Y[idx]

    for b in range(num_batches):
      s, e = b*batch_size, min((b+1)*batch_size, N)
      Xb = X_shuf[s:e]    # (B, d)
      Yb = Y_shuf[s:e]    # (B, k)

      h_hat = g(z(Xb, W_h, b_h))
      y_hat = g(z(h_hat, W_y, b_y))
      
      error = loss(Yb, y_hat)

      delta_y = error * dg(y_hat)
      delta_h = delta_y.dot(W_y.T) * dg(h_hat)
      
      W_h = W_h + alpha * Xb.T.dot(delta_h)
      W_y = W_y + alpha * h_hat.T.dot(delta_y)
      
      b_y = b_y + alpha * delta_y.sum(axis=0)
      b_h = b_h + alpha * delta_h.sum(axis=0)

    # Validamos sobre todo el conjunto
    h_hat = g(z(X, W_h, b_h))
    y_hat = g(z(h_hat, W_y, b_y))
    error = loss(Y, y_hat)
    mse = np.mean(error ** 2)
    losses.append(mse)

    if (t+1) % 5 == 0:
      print(f"Época {t+1}/{tmax} - Loss: {mse:.4f}")

  # Curva de error
  plt.figure(figsize=(6,3))
  plt.plot(losses)
  plt.xlabel("Época")
  plt.ylabel("MSE")
  plt.title("Entrenamiento")
  plt.show()

  return W_h, b_h, W_y, b_y

Ejecutá la celda siguiente para entrenarlo con el conjunto MNIST. Sé paciente, lleva tiempo ya que el entrenamiento se realiza en la CPU.

In [ ]:
W_h, b_h, W_r, b_r = perceptron_multicapa(X, Y, nodos_capa_oculta=256, tmax=25, alpha=0.01, batch_size=256)

@widgets.interact(i=(0, len(X)-1))
def inferencia(i):
  Xi = X[i]
  Yi = Y[i]
  
  plt.matshow(Xi.reshape(28, 28), cmap='gray')
  plt.show()
  
  h_hat = g(z(Xi, W_h, b_h))
  y_hat = g(z(h_hat, W_r, b_r))
  
  print("Salida inferida:", digits[np.argmax(y_hat)])
  print("Salida esperada:", Yi)